In [1]:
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from fastai.text import *

%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [36]:
bs = 48
path = untar_data(URLs.IMDB)

In [3]:
data_lm = load_data(path, 'data_lm.pkl', bs = bs)

In [4]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3)

### 0. Prior Inspection

In [5]:
len(data_lm.vocab.itos)

60000

In [6]:
learn.model.state_dict()['0.encoder.weight'].shape

torch.Size([60000, 400])

### 1. Prepare Test Data

In [ ]:
learn.data.vocab.numericalize(['good', 'nice', 'bad', 'horrible'])

In [9]:
w_mat = learn.model.state_dict()['0.encoder.weight']

In [13]:
def encode_word(idx, w_mat):
    """ direct index is fast and memory efficient """
    # pad one dim for sklearn computation
    return np.expand_dims(w_mat[idx].cpu().numpy(), axis = 0)

In [24]:
nice_emb = encode_word(359, w_mat)
good_emb = encode_word(68, w_mat)
horrible_emb = encode_word(503, w_mat)
bad_emb = encode_word(98, w_mat)

### 2. Test Cosine Similarity

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

In [29]:
cosine_similarity(nice_emb, bad_emb)

array([[0.335171]], dtype=float32)

In [35]:
cosine_similarity(horrible_emb, bad_emb)

array([[0.327662]], dtype=float32)